In [1]:
import cv2
import os,glob
import numpy as np
from time import sleep

In [2]:
#save image
def saveImg(image,index):
    filename='images/'+ name +'/face{:03d}.jpg'.format(index)
    cv2.imwrite(filename,image)

#take 100 pictures
index=1
total=100 

name=input("Name:")

if os.path.isdir('images/'+ name):
    print("The name has already been used!")
else:
    os.mkdir('images/'+ name) #create the file
    face_cascade=cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml")
    cap=cv2.VideoCapture(0)   #open the camera
    cv2.namedWindow("video",cv2.WINDOW_NORMAL) 
    while index>0:
        ret, frame = cap.read()     # Capture a frame from the webcam
        frame = cv2.flip(frame, 1)  # Flip the frame horizontally
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert the frame to grayscale
        faces = face_cascade.detectMultiScale(gray, 1.1, 3)
        for (x, y, w, h) in faces:
            frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)   # Draw a rectangle around the face
            image = cv2.resize(gray[y: y + h, x: x + w], (400, 400))               # Crop and resize the face region
            saveImg(image, index)  # Save the cropped face image
            sleep(0.1)             # Wait for 0.1 seconds
            index += 1
        if index > total:
            print("Sampling completed!")
            index=-1
            break
        cv2.imshow('video',frame)
        cv2.waitKey(1)
    cap.release()
    cv2.destroyAllWindows()

Name:Ivanlin
The name has already been used!


In [20]:
images = []             # List to store images
labels = []             # List to store labels for each image
labelstr = []           # List to store label strings (names)
count = 0               # Counter for labeling

dirs = os.listdir("images")           # Get list of directories in the "images" folder
for d in dirs:
    if os.path.isdir('images/' + d):                 # Check if the item is a directory
        files = glob.glob('images/' + d + '/*.jpg')  # Get all JPEG files in the directory
        for filename in files:                       # Iterate through all files in the directory
            img = cv2.imread(filename, cv2.COLOR_BGR2GRAY)  # Read the image in grayscale
            images.append(img)                       # Add the image to the images list
            labels.append(count)                     # Add the label (count) to the labels list
        labelstr.append(d)                           # Add the directory name as a label string
        count += 1                                   # Increment the label count



In [21]:
print('Start building Model...')
model = cv2.face.LBPHFaceRecognizer_create()  # Create an empty LBPH model
model.train(np.asarray(images), np.asarray(labels))  # Train the model with the images and labels
model.save('faces_LBPH.yml')  # Save the trained model to a file
print('Model creation completed!')

Start building Model...
Model creation completed!


In [26]:
f=open("member.txt",'w')
f.write(','.join(labelstr))
f.close()

In [27]:
import cv2
import time

In [37]:
import cv2
import time

# Initialize video capture from the default camera (0)
cap = cv2.VideoCapture(0)

# Create a named window with normal size
cv2.namedWindow('frame', cv2.WINDOW_NORMAL)

# Record the current time
timenow = time.time()

while True:
    # Calculate the countdown from 5 seconds
    count = 5 - int(time.time() - timenow)

    # Make a copy of the current image for drawing the countdown
    imgcopy = img.copy()

    # Draw the countdown number on the image
    cv2.putText(imgcopy, str(count), (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 15, (0, 0, 255), 35)

    # Read a frame from the camera
    ret, img = cap.read()

    # Flip the image horizontally for a mirror effect
    img = cv2.flip(img, 1)

    # Display the image with the countdown
    cv2.imshow("frame", imgcopy)

    # Wait for 100 milliseconds
    k = cv2.waitKey(100)

    # If 'z' or 'Z' is pressed, or the countdown reaches zero, save the image and break the loop
    if k == ord("z") or k == ord("Z") or count == 0:
        cv2.imwrite("tem.jpg", img)  # Save the captured image as "tem.jpg"
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [41]:
# Load the trained LBPH face recognizer model from the specified file
model = cv2.face.LBPHFaceRecognizer_create()  # Create an empty LBPH face recognizer model
model.read('faces_LBPH.yml')  # Load the trained model parameters from the YAML file

# Open the member names text file for reading
f = open('member.txt', 'r')  

# Read the first line from the file and split it into a list of names
# The names are expected to be separated by commas in the file
names = f.readline().split(',')  

# Close the file after reading
f.close()  


In [46]:
# Load the Haar Cascade classifier for frontal face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml")

# Read the image file where the face will be detected
img = cv2.imread("tem.jpg")

# Convert the image to grayscale for better face detection performance
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect faces in the image; the parameters control scale factor and min neighbors
faces = face_cascade.detectMultiScale(gray, 1.1, 3)

# Loop through all detected faces
for (x, y, w, h) in faces:
    # Draw a rectangle around each detected face
    img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 3)

    # Extract the face region and resize it to 400x400 pixels for recognition
    face_img = cv2.resize(gray[y: y + h, x: x + w], (400, 400))

    try:
        # Predict the identity of the detected face using the trained model
        val = model.predict(face_img)

        # Check the confidence level (val[1]) to determine if the prediction is reliable
        if val[1] < 50:  
            print('Welcome ' + names[val[0]], val[1])  # Print the recognized member's name
        else:
            print('Sorry! You are not a member, cannot access!')  # If confidence is too high, deny access
    except Exception as e:
        print('Error!', e)  # Catch any exceptions and print the error message


Welcome Ivanlin 25.151980076123937
